In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
        self.bn1 = nn.BatchNorm2d(16)
        self.do1 = nn.Dropout2d(0.1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.do2 = nn.Dropout2d(0.1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.transition1 = nn.Conv2d(32, 16, kernel_size=1)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(16)
        self.do3 = nn.Dropout2d(0.1)
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(16)
        self.do4 = nn.Dropout2d(0.1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.transition2 = nn.Conv2d(16, 8, kernel_size=1)
        self.conv5 = nn.Conv2d(8, 8, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(8)
        self.do5 = nn.Dropout2d(0.1)
        self.conv6 = nn.Conv2d(8, 8, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(8)
        self.do6 = nn.Dropout2d(0.1)
        self.fc1 = nn.Linear(8 * 7 * 7, 21)  # Assuming MNIST input size of 28x28
        self.fc2 = nn.Linear(21, 10)

    def forward(self, x):
        x = self.pool1(self.do2(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x))))))))
        x = self.transition1(x)
        x = self.pool2(self.do4(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x))))))))
        x = self.transition2(x)
        x = self.do6(self.bn6(F.relu(self.conv6(self.bn5(F.relu(self.conv5(x)))))))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 32, 28, 28]           4,640
       BatchNorm2d-4           [-1, 32, 28, 28]              64
         Dropout2d-5           [-1, 32, 28, 28]               0
         MaxPool2d-6           [-1, 32, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]             528
            Conv2d-8           [-1, 16, 14, 14]           2,320
       BatchNorm2d-9           [-1, 16, 14, 14]              32
           Conv2d-10           [-1, 16, 14, 14]           2,320
      BatchNorm2d-11           [-1, 16, 14, 14]              32
        Dropout2d-12           [-1, 16, 14, 14]               0
        MaxPool2d-13             [-1, 16, 7, 7]               0
           Conv2d-14              [-1, 

In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.15MB/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 57.3kB/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:06<00:00, 245kB/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.05MB/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    if((100*(float(correct) / float(len(test_loader.dataset)))) > 99.4):
      torch.save(model.state_dict(), "models\model.pth")

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

In [8]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1,cooldown=5)

for epoch in range(1, 20):
    print("## Epoch " + str(epoch) + ":")
    train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader)
    scheduler.step(test_loss)
    print(f"Learning Rate = {optimizer.param_groups[0]['lr']}\n")

## Epoch 1:


loss=0.017595084384083748 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.63it/s]



Test set: Average loss: 0.0539, Accuracy: 9825/10000 (98%)

Learning Rate = 0.01

## Epoch 2:


loss=0.11072218418121338 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.29it/s]



Test set: Average loss: 0.0509, Accuracy: 9828/10000 (98%)

Learning Rate = 0.01

## Epoch 3:


...


Test set: Average loss: 0.0182, Accuracy: 9942/10000 (99%)

Learning Rate = 0.0001

## Epoch 18:


loss=0.00746827432885766 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.94it/s]



Test set: Average loss: 0.0183, Accuracy: 9941/10000 (99%)

Learning Rate = 0.0001

## Epoch 19:


loss=0.06835322827100754 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.18it/s]



Test set: Average loss: 0.0183, Accuracy: 9941/10000 (99%)

Learning Rate = 0.0001

